# Example for Qwen3-VL-Reranker using vLLM

## 1. Import Packages and Prepare Utility Functions

In [1]:
import os
from pathlib import Path
from typing import Dict, Any
from jinja2 import Template
from vllm import LLM
from vllm.multimodal.utils import fetch_image

def parse_input_dict(input_dict: Dict[str, Any]):
    """
    Parse input dictionary to extract image and text content.
    Returns the formatted content string and multimodal data.
    """
    image = input_dict.get('image')
    text = input_dict.get('text')

    mm_data = {
        'image': []
    }
    content = ''
    if image:
        content += '<|vision_start|><|image_pad|><|vision_end|>'
        if isinstance(image, str):
            if image.startswith(('http://', 'https://')):
                try:
                    image_obj = fetch_image(image)
                    mm_data['image'].append(image_obj)
                except Exception as e:
                    print(f"Warning: Failed to fetch image {image}: {e}")
            else:
                abs_image_path = os.path.abspath(image)
                if os.path.exists(abs_image_path):
                    from PIL import Image
                    image_obj = Image.open(abs_image_path)
                    mm_data['image'].append(image_obj)
                else:
                    print(f"Warning: Image file not found: {abs_image_path}")
        else:
            mm_data['image'].append(image)
    
    if text:
        content += text
    
    return content, mm_data

def format_vllm_input(
    query_dict: Dict[str, Any],
    doc_dict: Dict[str, Any],
    chat_template: str
):
    """
    Format query and document into vLLM input format.
    Combines multimodal data from both query and document.
    """
    query_content, query_mm_data = parse_input_dict(query_dict)
    doc_content, doc_mm_data = parse_input_dict(doc_dict)

    mm_data = { 'image': [] }
    mm_data['image'].extend(query_mm_data['image'])
    mm_data['image'].extend(doc_mm_data['image'])

    prompt = Template(chat_template).render(
        query_content=query_content,
        doc_content=doc_content,
    )
    return {
        'prompt': prompt,
        'multi_modal_data': mm_data
    }

def get_rank_scores(
    llm,
    inputs: Dict[str, Any],
    default_instruction: str = "Given a search query, retrieve relevant candidates that answer the query.",
    template_path: str = "reranker_template.jinja"
):
    """
    Generate relevance scores for documents given a query.
    Returns a list of scores for each document.
    """
    query_dict = inputs['query']
    doc_dicts = inputs['documents']
    instruction = inputs.get('instruction') or default_instruction

    chat_template = Template(Path(template_path).read_text())
    chat_template = chat_template.render(instruction=instruction)

    prompts = []

    for doc_dict in doc_dicts:
        prompt = format_vllm_input(
            query_dict, doc_dict, chat_template
        )
        prompts.append(prompt)

    outputs = llm.classify(
        prompts=prompts
    )
    scores = [ output.outputs.probs[0] for output in outputs ]
    return scores

/cpfs01/user/linqi.lmx/envs/vllm014n/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Initialize Model

In [2]:
# Initialize the Qwen3-VL-Reranker model
llm = LLM(
    model='/cpfs01/user/linqi.lmx/models/finetune/qwen/qwen3-vl/Qwen3-VL-Embedding/Qwen3-VL-Reranker-2B',
    runner='pooling',
    dtype='bfloat16',
    trust_remote_code=True,
    hf_overrides={
        "architectures": ["Qwen3VLForSequenceClassification"],
        "classifier_from_token": ["no", "yes"],
        "is_original_qwen3_reranker": True,
    },
)

print("Model initialized successfully!")

INFO 01-16 16:15:13 [utils.py:267] non-default args: {'runner': 'pooling', 'trust_remote_code': True, 'dtype': 'bfloat16', 'disable_log_stats': True, 'hf_overrides': {'architectures': ['Qwen3VLForSequenceClassification'], 'classifier_from_token': ['no', 'yes'], 'is_original_qwen3_reranker': True}, 'model': '/cpfs01/user/linqi.lmx/models/finetune/qwen/qwen3-vl/Qwen3-VL-Embedding/Qwen3-VL-Reranker-2B'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 01-16 16:15:13 [model.py:859] Resolved `--convert auto` to `--convert classify`. Pass the value explicitly to silence this message.


INFO 01-16 16:15:13 [model.py:530] Resolved architecture: Qwen3VLForSequenceClassification


...

INFO 01-16 16:15:56 [gpu_model_runner.py:4852] Graph capturing finished in 3 secs, took -0.76 GiB


(EngineCore_DP0 pid=200152) 

INFO 01-16 16:15:56 [core.py:272] init engine (profile, create kv cache, warmup model) took 25.93 seconds


INFO 01-16 16:16:03 [llm.py:347] Supported tasks: ['classify', 'score', 'token_classify']


Model initialized successfully!


## 3. Prepare Input Data

In [3]:
# Define query and candidate documents for reranking
inputs = {
    "instruction": "Retrieve images or text relevant to the user's query.",
    "query": {
        "text": "A woman playing with her dog on a beach at sunset."
    },
    "documents": [
        {
            "text": "A woman shares a joyful moment with her golden retriever on a sun-drenched beach at sunset, as the dog offers its paw in a heartwarming display of companionship and trust."
        },
        {
            "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
        },
        {
            "text": "A woman shares a joyful moment with her golden retriever on a sun-drenched beach at sunset, as the dog offers its paw in a heartwarming display of companionship and trust.",
            "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
        }
    ]
}

print(f"Prepared query with {len(inputs['documents'])} candidate documents")

Prepared query with 3 candidate documents


## 4. Generate Relevance Scores

In [4]:
# Get relevance scores for each document
scores = get_rank_scores(llm, inputs)

print("Relevance Scores:")
for i, score in enumerate(scores):
    print(f"Document {i+1}: {score:.4f}")

Adding requests:   0%|                                                                                                 | 0/3 [00:00<?, ?it/s]

Adding requests:  33%|█████████████████████████████▋                                                           | 1/3 [00:02<00:04,  2.40s/it]

Adding requests: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.51it/s]

Adding requests: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.20it/s]

Processed prompts:   0%|                                           | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|███████████████████████████████████| 3/3 [00:00<00:00, 48.83it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|███████████████████████████████████| 3/3 [00:00<00:00, 48.48it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Relevance Scores:
Document 1: 0.8600
Document 2: 0.7232
Document 3: 0.8189
